# Yelp API - Lab


## Introduction 

We've seen how the Yelp API works and how to create basic visualizations using Folium. It's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 

* Using pagination and multiple functions, gather large amounts of data from an API, parse the data and make sense of it with meaningful analysis
* Create maps using Folium

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you but make the request below.

In [1]:
# Your code here
import requests
import pandas as pd
import json

with open('/Users/jamesirving/.secret/yelp_api.json') as f:
    creds = json.load(f)

In [2]:
url = 'https://api.yelp.com/v3/businesses/search'
term = 'Hamburgers'
SEARCH_LIMIT = 10

headers = {
    'Authorization': 'Bearer ' + creds['api_key']
}

url_params = {
    'term': term,
    'location': 'Seattle+WA',
    'limit': SEARCH_LIMIT,
    'offset': 0
}

response = requests.get(url, headers=headers, params=url_params)
print(response.status_code)

200


In [10]:
def get_results(term,location,  offset = 0,
                url = 'https://api.yelp.com/v3/businesses/search',
               cred=None,fpath='/Users/jamesirving/.secret/yelp_api.json'):
#     url = 'https://api.yelp.com/v3/businesses/search'

    if cred is None:
        with open(fpath) as f:
            cred = json.load(f)


    headers = {
        'Authorization': 'Bearer ' + cred['api_key']
    }

    url_params = {
        'term': term,
        'location': location,
#         'limit': SEARCH_LIMIT,
        'offset': offset
    }

    response = requests.get(url, headers=headers, params=url_params)
    return response.json()

In [11]:
response = get_results('Chinese','Baltimore MD')
response.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
response['total']

335

In [14]:
n_per_page = len(response['businesses'])
n_per_page

20

In [17]:
pd.DataFrame(response['businesses']).tail()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
15,m69hGqYXyn2RZM_Sl7-ciQ,panda-bbq-baltimore,Panda BBQ,https://s3-media2.fl.yelpcdn.com/bphoto/D8aOHy...,False,https://www.yelp.com/biz/panda-bbq-baltimore?a...,54,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.5,"{'latitude': 39.303624, 'longitude': -76.617175}","[delivery, pickup]",NaN,"{'address1': '30 W Biddle St', 'address2': Non...",+13108943736,(310) 894-3736,941.481971
16,3Szif3O7-yN90I_SEkxWNw,red-pepper-sichuan-bistro-towson,Red Pepper Sichuan Bistro,https://s3-media2.fl.yelpcdn.com/bphoto/0Lzq6k...,False,https://www.yelp.com/biz/red-pepper-sichuan-bi...,107,"[{'alias': 'szechuan', 'title': 'Szechuan'}]",4.5,"{'latitude': 39.40145, 'longitude': -76.6032}","[delivery, pickup]",$$,"{'address1': '11 Allegheny Ave', 'address2': N...",+14108327333,(410) 832-7333,10260.957618
17,7jdUowVElWm5IFylVpj8yQ,blessings-chinese-and-american-food-carry-out-...,Blessings Chinese & American Food Carry Out,https://s3-media1.fl.yelpcdn.com/bphoto/pOmogU...,False,https://www.yelp.com/biz/blessings-chinese-and...,13,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 39.2972468809007, 'longitude': -7...",[delivery],$,"{'address1': '537 N Chester St', 'address2': '...",+14103270258,(410) 327-0258,2264.017853
18,5rYJkTY_V5y5IGepcDmW_w,bambao-baltimore,Bambao,https://s3-media4.fl.yelpcdn.com/bphoto/m7bhGj...,False,https://www.yelp.com/biz/bambao-baltimore?adju...,79,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.0,"{'latitude': 39.28348, 'longitude': -76.599289}","[delivery, pickup]",$$,"{'address1': '1010 Aliceanna St', 'address2': ...",+14434383528,(443) 438-3528,2989.663642
19,W2ZYDx9kHdgDCRyMf2cj5g,yungs-carry-out-baltimore,Yung's Carry Out,https://s3-media4.fl.yelpcdn.com/bphoto/RbdaI2...,False,https://www.yelp.com/biz/yungs-carry-out-balti...,27,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.5,"{'latitude': 39.29932, 'longitude': -76.61198}","[delivery, pickup]",$,"{'address1': '818 Guilford Ave', 'address2': '...",+14105398200,(410) 539-8200,1167.404405


In [18]:
response2 = get_results('Chinese','Baltimore MD',offset=n_per_page)
pd.DataFrame(response2['businesses']).head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,lX7xaoJy7ikZWuACkflWOQ,china-wok-baltimore-3,China Wok,https://s3-media4.fl.yelpcdn.com/bphoto/cFHHoc...,False,https://www.yelp.com/biz/china-wok-baltimore-3...,13,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.0,"{'latitude': 39.295040727015, 'longitude': -76...",[delivery],$$,"{'address1': '200 N Highland Ave', 'address2':...",+14107323818,(410) 732-3818,3719.956410
1,vOvqK5zBxR1xfXpQUGto3A,asian-harbor-baltimore,Asian Harbor,https://s3-media3.fl.yelpcdn.com/bphoto/k9mzuh...,False,https://www.yelp.com/biz/asian-harbor-baltimor...,140,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.5,"{'latitude': 39.28916, 'longitude': -76.60202}","[delivery, pickup]",$,"{'address1': '1007 E Lombard St', 'address2': ...",+14103276688,(410) 327-6688,2316.690700
2,KqLOev35hJ9H9jx4Fgu-bg,chow-mein-charlie-baltimore,Chow Mein Charlie,https://s3-media3.fl.yelpcdn.com/bphoto/M38S3j...,False,https://www.yelp.com/biz/chow-mein-charlie-bal...,35,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.5,"{'latitude': 39.3446732, 'longitude': -76.6313...",[delivery],$$,"{'address1': '508 W Cold Spring Ln', 'address2...",+14108898886,(410) 889-8886,4375.615880
3,EyzwOeUWwoKGeZcOcVUW0A,ekiben-baltimore-3,Ekiben,https://s3-media3.fl.yelpcdn.com/bphoto/Wcriq5...,False,https://www.yelp.com/biz/ekiben-baltimore-3?ad...,1247,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.5,"{'latitude': 39.2857429, 'longitude': -76.594557}",[delivery],$$,"{'address1': '1622 Eastern Ave', 'address2': '...",,,2899.376967
4,Daqo6Vkj-m42dMZghpM6fQ,chang-le-baltimore,Chang Le,https://s3-media3.fl.yelpcdn.com/bphoto/zW9a6l...,False,https://www.yelp.com/biz/chang-le-baltimore?ad...,2,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.5,"{'latitude': 39.3563004, 'longitude': -76.6097...","[delivery, pickup]",NaN,"{'address1': '5415 York Rd', 'address2': '', '...",+14104640880,(410) 464-0880,5216.715682


## Pagination

Now that you have an initial response, you can examine the contents of the JSON container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [20]:
# Your code here; use a function or loop to retrieve all the results from your original request
offset=0
response = get_results('Chinese','Baltimore MD',offset=offset)

n_per_page = len(response['businesses'])
total_result = response['total']

results = response['businesses']

while offset < total_result:
    offset+=n_per_page
    res = get_results('Chinese','Baltimore MD',offset=offset)
    results.extend(res['businesses'])
results[:5]

[{'id': 'ry3Y5KSxtn7rVf3amw31rw',
  'alias': 'china-house-baltimore-2',
  'name': 'China House',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/EjebY0vjN0gFxoCCWJJMGQ/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/china-house-baltimore-2?adjust_creative=dTD4bQ8glEIuLwnjjDnCgQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=dTD4bQ8glEIuLwnjjDnCgQ',
  'review_count': 184,
  'categories': [{'alias': 'chinese', 'title': 'Chinese'}],
  'rating': 4.0,
  'coordinates': {'latitude': 39.2856979370117,
   'longitude': -76.5847778320312},
  'transactions': ['delivery'],
  'price': '$',
  'location': {'address1': '2233 Eastern Ave',
   'address2': '',
   'address3': '',
   'city': 'Baltimore',
   'zip_code': '21231',
   'country': 'US',
   'state': 'MD',
   'display_address': ['2233 Eastern Ave', 'Baltimore, MD 21231']},
  'phone': '+14105223456',
  'display_phone': '(410) 522-3456',
  'distance': 3344.55380067889},
 {'id': 'JZlGOTvnm2a8B7eGRlZZUg',

In [23]:
df = pd.DataFrame(results)
df.to_csv('yelp_reslts.csv',index=False)
df = pd.read_csv('yelp_reslts.csv')
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ry3Y5KSxtn7rVf3amw31rw,china-house-baltimore-2,China House,https://s3-media3.fl.yelpcdn.com/bphoto/EjebY0...,False,https://www.yelp.com/biz/china-house-baltimore...,184,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.0,"{'latitude': 39.2856979370117, 'longitude': -7...",['delivery'],$,"{'address1': '2233 Eastern Ave', 'address2': '...",1.410522e+10,(410) 522-3456,3344.553801
1,JZlGOTvnm2a8B7eGRlZZUg,nihao-baltimore-2,NiHao,https://s3-media1.fl.yelpcdn.com/bphoto/i5Vmne...,False,https://www.yelp.com/biz/nihao-baltimore-2?adj...,80,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.5,"{'latitude': 39.282945, 'longitude': -76.583924}",['delivery'],NaN,"{'address1': '2322 Boston St', 'address2': '',...",1.443835e+10,(443) 835-2036,3640.045127
2,ZFFQuoUajWY_gYZ4HjfIKQ,water-song-yunnan-kitchen-baltimore-3,Water Song Yunnan Kitchen,https://s3-media1.fl.yelpcdn.com/bphoto/lL_yo7...,False,https://www.yelp.com/biz/water-song-yunnan-kit...,68,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.5,"{'latitude': 39.276528, 'longitude': -76.613502}","['delivery', 'pickup']",NaN,"{'address1': '21 E Cross St', 'address2': '', ...",1.443869e+10,(443) 869-4580,3664.817209
3,C6ktcKtaj62akb8X0CqAWg,sunny-carry-out-baltimore,Sunny Carry Out,https://s3-media2.fl.yelpcdn.com/bphoto/uTMvf4...,False,https://www.yelp.com/biz/sunny-carry-out-balti...,76,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.0,"{'latitude': 39.28325, 'longitude': -76.62783}",['pickup'],$,"{'address1': '758 Washington Blvd', 'address2'...",1.410245e+10,(410) 244-8816,3317.393328
4,Mt8KF8_1T27QAY3ol1S2cg,wow-food-cafe-baltimore-9,WOW Food Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/O8J8BF...,False,https://www.yelp.com/biz/wow-food-cafe-baltimo...,177,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...",4.5,"{'latitude': 39.292012, 'longitude': -76.616007}",['delivery'],$$,"{'address1': '222 N Charles St', 'address2': '...",1.667213e+10,(667) 213-0043,2012.195728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,UQ6BeOKeM-01v9-8-IQNhw,panera-bread-pikesville,Panera Bread,https://s3-media4.fl.yelpcdn.com/bphoto/LnZlJS...,False,https://www.yelp.com/biz/panera-bread-pikesvil...,37,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 39.3902676, 'longitude': -76.7412...",['pickup'],$,"{'address1': '1852 Reisterstown Rd', 'address2...",1.410603e+10,(410) 602-5125,14504.936752
331,AtwKjW3qylM0781037XMEQ,noodles-and-company-columbia,Noodles & Company,https://s3-media3.fl.yelpcdn.com/bphoto/iMMLXx...,False,https://www.yelp.com/biz/noodles-and-company-c...,135,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",3.0,"{'latitude': 39.1994255, 'longitude': -76.81267}",[],$,"{'address1': '6191 Old Dobbin Ln', 'address2':...",1.410872e+10,(410) 872-3729,21369.206243
332,K1Xgb1aTIqL7qD0MjewpAA,kellys-cajun-grill-hanover,Kelly's Cajun Grill,https://s3-media2.fl.yelpcdn.com/bphoto/HuOdld...,False,https://www.yelp.com/biz/kellys-cajun-grill-ha...,26,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",2.5,"{'latitude': 39.1570476964906, 'longitude': -7...",[],$,"{'address1': '7000 Arundel Mills Cir', 'addres...",1.410541e+10,(410) 540-5110,19684.713837
333,viOIpHVViYTOtJw8C7UXng,pho-viet-baltimore-3,Pho Viet,https://s3-media1.fl.yelpcdn.com/bphoto/k5OjG8...,False,https://www.yelp.com/biz/pho-viet-baltimore-3?...,157,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"{'latitude': 39.290687, 'longitude': -76.614278}","['pickup', 'delivery']",$$,"{'address1': '104 Saint Paul St', 'address2': ...",1.410244e+10,(410) 244-1428,2116.690511


In [24]:
def get_lat_long(row):
    coordinates = row['coordinates']
    row['lat'] = coordinates['latitude']
    row['lon'] =coordinates['longitude']
    return row

In [33]:
with open('yelp_results.json','w') as f:
    json.dump(results,f)

In [34]:
with open('yelp_results.json','r') as f:
    data = json.loads(f.read())
data

[{'id': 'ry3Y5KSxtn7rVf3amw31rw',
  'alias': 'china-house-baltimore-2',
  'name': 'China House',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/EjebY0vjN0gFxoCCWJJMGQ/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/china-house-baltimore-2?adjust_creative=dTD4bQ8glEIuLwnjjDnCgQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=dTD4bQ8glEIuLwnjjDnCgQ',
  'review_count': 184,
  'categories': [{'alias': 'chinese', 'title': 'Chinese'}],
  'rating': 4.0,
  'coordinates': {'latitude': 39.2856979370117,
   'longitude': -76.5847778320312},
  'transactions': ['delivery'],
  'price': '$',
  'location': {'address1': '2233 Eastern Ave',
   'address2': '',
   'address3': '',
   'city': 'Baltimore',
   'zip_code': '21231',
   'country': 'US',
   'state': 'MD',
   'display_address': ['2233 Eastern Ave', 'Baltimore, MD 21231']},
  'phone': '+14105223456',
  'display_phone': '(410) 522-3456',
  'distance': 3344.55380067889},
 {'id': 'JZlGOTvnm2a8B7eGRlZZUg',

In [35]:
# df.apply(get_lat_long,axis=1)

In [27]:
df.loc[0,'coordinates']

"{'latitude': 39.2856979370117, 'longitude': -76.5847778320312}"

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
# Your code here

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
# Your code here

## Summary

Nice work! In this lab, you've made multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice interactive map to display the results using Folium! Well done!